In [ ]:
import sys, os 
import ipywidgets as widgets
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent.parent))
from demoTools.demoutils import *
from IPython.display import display

In [ ]:
try: 
    import keras
except:
    print("Keras not installed")
    !{sys.executable} -m pip install keras

In [ ]:
try: 
    import psutil
except:
    print("psutil not installed")
    !{sys.executable} -m pip install psutil

## Create an Intermediate Representation (IR) Model using the Model Optimizer by Intel

The Model Optimizer creates Intermediate Representation (IR) models that are optimized for different end-point target devices.
These models can be created from existing DNN models from popular frameworks (e.g. Caffe*, TF) using the Model Optimizer. 

The Intel® Distribution of OpenVINO™ toolkit includes a utility script `model_downloader.py` that you can use to download some common modes. Run the following cell to see the models available through `model_downloader.py`

In [ ]:
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --print_all

To download model using the model downloader you will need to run the following command:
`!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name <model name> -o <output directory>`

The input arguments are as follows:
* --name : name of the model you want to download. It should be one of the models listed in the previous cell
* -o : output directory. If this directory does not exist, it will be created for you.

There are more arguments to this script and you can get the full list using the `-h` option.

Now, let's convert the model to the optimized model using the model optimizer.

`!/opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
--input_model <model> \
--data_type FP32 \
-o <output directory> `

**Note** the above line is a single command line input, which spans 4 lines thanks to the backslash '\\', which is a line continuation character in Bash.

Here, the arguments are:
* --input-model : the original model
* --data_type : Data type to use. One of {FP32, FP16, half, float}
* -o : output directory (If this directory does not exist, it will be created for you.)

This script also supports `-h` that will you can get the full list of arguments.

Running that command will produce two files:
```
<output directory>/<model>.xml
<output directory>/<model>.bin
```
These will be used later in the exercise.

We will also be needing the FP16 version of the model for the calculations on the VPU architecture. Run the following command to create it.

`!/opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
--input_model <model> \
--data_type FP16 \
-o <output directory> `

In [ ]:
%%writefile healthcare_job_openvino.sh

cd $PBS_O_WORKDIR
DEVICE=$1
RESULTS=$2

if [ "$DEVICE" = "HETERO:FPGA,CPU" ]; then
    # Environment variables and compilation for edge compute nodes with FPGAs
    source /opt/fpga_support_files/setup_env.sh
    aocl program acl0 /opt/intel/openvino/bitstreams/a10_vision_design_bitstreams/2019R1_PL1_FP11_MobileNet_Clamp.aocx
fi
     
# Running the object detection code
SAMPLEPATH=$PBS_O_WORKDIR
python3 healthcare_openvino.py     -d $DEVICE \
                                   -l /opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/libcpu_extension_avx2.so \
                                   -r $RESULTS


#### Submitting to an edge compute node with an Intel® CPU
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank* 870-Q170</a> edge node with an <a 
    href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel® Core™ i5-6500TE processor</a>. The inference workload will run the CPU.

In [ ]:
job_id_core = !qsub healthcare_job_openvino.sh -l nodes=1:tank-870:i5-6500te  -F "CPU results/"
print(job_id_core[0]) 
if job_id_core:
    progressIndicator('results/'+job_id_core[0], 'i_progress.txt', "Processing", 0, 100)

#### Submitting to an edge compute node with Intel® Xeon® CPU
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank* 870-Q170</a> edge node with an <a 
    href="https://ark.intel.com/products/88178/Intel-Xeon-Processor-E3-1268L-v5-8M-Cache-2-40-GHz-">Intel® 
    Xeon® Processor E3-1268L v5</a>. The inference workload will run on the CPU.

In [ ]:
job_id_xeon = !qsub healthcare_job_openvino.sh -l nodes=1:tank-870:e3-1268l-v5  -F "CPU results/"
print(job_id_xeon[0]) 
if job_id_xeon:
    progressIndicator('results/'+job_id_xeon[0], 'i_progress.txt', "Processing", 0, 100)

#### Submitting to an edge compute node with Intel® Core CPU and using the onboard Intel® GPU
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank* 870-Q170</a> edge node with an <a href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel® Core i5-6500TE</a>. The inference workload will run on the Intel® HD Graphics 530 card integrated with the CPU.

In [ ]:
#Submit job to the queue
job_id_gpu = !qsub healthcare_job_openvino.sh -l nodes=1:tank-870:i5-6500te:intel-hd-530 -F "GPU results/"
print(job_id_gpu) 
if job_id_gpu:
    progressIndicator('results/'+job_id_gpu[0], 'i_progress.txt', "Processing", 0, 100)

#### Submitting to an edge compute node with Intel® NCS 2 (Neural Compute Stick 2)
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel Core i5-6500te CPU</a>. The inference workload will run on an <a 
    href="https://software.intel.com/en-us/neural-compute-stick">Intel Neural Compute Stick 2</a> installed in this  node.

In [ ]:
#Submit job to the queue
job_id_ncs2 = !qsub healthcare_job_openvino.sh -l nodes=1:tank-870:i5-6500te:intel-ncs2 -F "MYRIAD results/"
print(job_id_ncs2[0]) 
if job_id_ncs2:
    progressIndicator('results/'+job_id_ncs2[0], 'i_progress.txt', "Processing", 0, 100)

#### Submitting to an edge compute node with Intel® NCS 2 (Neural Compute Stick 2)
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel Core i5-6500te CPU</a>. The inference workload will run on an <a 
    href="https://software.intel.com/en-us/neural-compute-stick">Intel Neural Compute Stick 2</a> installed in this  node.

In [ ]:
job_id_hddlr = !qsub healthcare_job_openvino.sh -l nodes=1:tank-870:i5-6500te:iei-mustang-v100-mx8 -F "HDDL results/"
print(job_id_hddlr[0]) 
if job_id_hddlr:
    progressIndicator('results/'+job_id_hddlr[0], 'i_progress.txt', "Processing", 0, 100)

In [ ]:
liveQstat()

In [ ]:
outputHTML('IEI Tank (Intel Core CPU)',
          'results/'+job_id_core[0], '.png')

In [ ]:
outputHTML('IEI Tank Xeon (Intel Xeon CPU)',
          'results/'+job_id_xeon[0], '.png')

In [ ]:
outputHTML('IEI Intel GPU (Intel Core + Onboard GPU)',
          'results/'+job_id_gpu[0], '.png')

In [ ]:
outputHTML('IEI Tank + Intel CPU + Intel NCS2',
          'results/'+job_id_ncs2[0], '.png')

In [ ]:
outputHTML('IEI Tank + IEI Mustang-V100-MX8 ( Intel® Movidius™ Myriad™ X Vision Processing Unit (VPU))',
          'results/'+job_id_hddlr[0], '.png')

In [ ]:
arch_list = [('core', 'Intel Core\ni5-6500TE\nCPU'),
             ('xeon', 'Intel Xeon\nE3-1268L v5\nCPU'),
             ('gpu', ' Intel Core\ni5-6500TE\nGPU'),
             ('ncs2', 'Intel\nNCS2'),
             ('hddlr', ' IEI Mustang\nV100-MX8\nVPU')]

stats_list = []
for arch, a_name in arch_list:
    if 'job_id_'+arch in vars():
        stats_list.append(('results/'+vars()['job_id_'+arch][0]+'/stats.txt', a_name))
    else:
        stats_list.append(('placeholder'+arch, a_name))

summaryPlot(stats_list, 'Architecture', 'Time, seconds', 'Inference Engine Processing Time', 'time' )

summaryPlot(stats_list, 'Architecture', 'Frames per second', 'Inference Engine FPS', 'fps' )